In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
data_path = Path("../data/raw/CPFL-RS-Informações industrial grupo A.xlsx")
data = pd.read_excel(data_path)
data.head()

In [ ]:
start_row = 3

data_vars = data.iloc[start_row:,0:7]
data_vars.columns = data_vars.iloc[0,:].values
data_vars = data_vars.drop(start_row)
# data_vars = data_vars.reset_index(drop=True)
data_vars

In [ ]:
data.iloc[:,7:19]

Split data into headers and values

In [ ]:
data_header = data.iloc[]